# 1. Introducción e Importación de librerías y datos
### En este cuaderno vamos a practicar con diferentes algoritmos de clasificación:
* SVM(Support Vector Machine). En español: Máquina de vectores de soporte 
* Regresión Logística 
* Árboles de Decisión 
* Bosques aleatorios 
* kNN, o el algoritmo de k vecino más cercano
### Para los siguientes datasets:
* Phishing Urls
* Calidad del Vino(Si es 5 o menor es de mala calidad y si es mayor de 5 es buena calidad)
* Diagnostico de Cancer de Mama
* Temperatura en España (dias de lluvia de un mes y lugar concreto)

### Para cada algoritmo vamos a:
* Explicar en qué consiste el algoritmo
* Explicar los resultados en cada dataset en base a las métricas iniciales sin incluir hiperparámetros
* Ajuste de hiperparamétros en cada uno de los ejemplos (3 casos por cada hiperparámetros y comparar y explicar cuál es el mejor resultado) y explicar en que consisten cada uno de ellos y cuando usarlos.
* Analizar los resultados de cada dataset indicando cual es mejor algoritmo en cada caso
* Incluir dos nuevas métricas en cada algoritmo, explicar la métrica y analizar los resultados.

### Por cada dataset vamos a:
* Tratar los datos(limpieza, encoding y escalado) cuando sea necesario
* Crear uno de ejemplo con sólo 10 registros, con 5 para un caso y otros 5 para otro. Usar los algoritmos con estos dataset, con la configuración de hiperparametros que hayais obtenido mejor resultado e indicar en cada caso que resultado habéis obtenido.

### Imports necesarios

In [293]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import  OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR, SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score, f1_score, recall_score, precision_score, roc_auc_score, average_precision_score, explained_variance_score
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor

### Importamos los datasets

In [294]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

# PHISHING URLS
phishing_path = "phishing.csv"  # Ruta donde deseas guardar el dataset
# Cargar la última versión del dataset
dfphishing = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,  # Formato que queremos (Pandas)
  "eswarchandt/phishing-website-detector",  # Nombre del dataset
  phishing_path
)
# Mostrar las primeras 5 filas del dataset vino
display("DATASET PHISHING",dfphishing.head(5))

# CALIDAD DEL VINO
vino_path = "WineQT.csv"  # Ruta donde deseas guardar el dataset
# Cargar la última versión del dataset
dfvino = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,  # Formato que queremos (Pandas)
  "yasserh/wine-quality-dataset",  # Nombre del dataset
  vino_path
)
# Mostrar las primeras 5 filas del dataset vino
display("DATASET VINO",dfvino.head(5))

# CANCER DE MAMA
cancer_path = "brca.csv"  # Ruta donde deseas guardar el dataset
# Cargar la última versión del dataset
dfcancer = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,  # Formato que queremos (Pandas)
  "utkarshx27/breast-cancer-wisconsin-diagnostic-dataset",  # Nombre del dataset
  cancer_path
)
# Mostrar las primeras 5 filas del dataset cancer
display("DATASET CANCER",dfcancer.head(5))


# TEMPERATURA ESPAÑA
temp_path = "monthly_temperature_spain (1996-2023).csv"  # Ruta donde deseas guardar el dataset
# Cargar la última versión del dataset
dftemp = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,  # Formato que queremos (Pandas)
  "alexgczs/monthly-temperature-in-spain-1996-2023",  # Nombre del dataset
  temp_path
)
# Mostrar las primeras 5 filas del dataset temp españa
display("TEMPERATURA ESPAÑA",dftemp.head(5))

C:\Users\domid\AppData\Local\Temp\ipykernel_5404\2763246943.py:7: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  dfphishing = kagglehub.load_dataset(


'DATASET PHISHING'

,Index,UsingIP,LongURL,ShortURL,Symbol@,Redirecting//,PrefixSuffix-,SubDomains,HTTPS,DomainRegLen,...,UsingPopupWindow,IframeRedirection,AgeofDomain,DNSRecording,WebsiteTraffic,PageRank,GoogleIndex,LinksPointingToPage,StatsReport,class
0,0,1,1,1,1,1,-1,0,1,-1,...,1,1,-1,-1,0,-1,1,1,1,-1
1,1,1,0,1,1,1,-1,-1,-1,-1,...,1,1,1,-1,1,-1,1,0,-1,-1
2,2,1,0,1,1,1,-1,-1,-1,1,...,1,1,-1,-1,1,-1,1,-1,1,-1
3,3,1,0,-1,1,1,-1,1,1,-1,...,-1,1,-1,-1,0,-1,1,1,1,1
4,4,-1,0,-1,1,-1,-1,1,1,-1,...,1,1,1,1,1,-1,1,-1,-1,1


C:\Users\domid\AppData\Local\Temp\ipykernel_5404\2763246943.py:18: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  dfvino = kagglehub.load_dataset(


'DATASET VINO'

...

'DATASET CANCER'

,Unnamed: 0,x.radius_mean,x.texture_mean,x.perimeter_mean,x.area_mean,x.smoothness_mean,x.compactness_mean,x.concavity_mean,x.concave_pts_mean,x.symmetry_mean,...,x.texture_worst,x.perimeter_worst,x.area_worst,x.smoothness_worst,x.compactness_worst,x.concavity_worst,x.concave_pts_worst,x.symmetry_worst,x.fractal_dim_worst,y
0,1,13.540,14.36,87.46,566.3,0.09779,0.08129,0.06664,0.047810,0.1885,...,19.26,99.70,711.2,0.14400,0.17730,0.23900,0.12880,0.2977,0.07259,B
1,2,13.080,15.71,85.63,520.0,0.10750,0.12700,0.04568,0.031100,0.1967,...,20.49,96.09,630.5,0.13120,0.27760,0.18900,0.07283,0.3184,0.08183,B
2,3,9.504,12.44,60.34,273.9,0.10240,0.06492,0.02956,0.020760,0.1815,...,15.66,65.13,314.9,0.13240,0.11480,0.08867,0.06227,0.2450,0.07773,B
3,4,13.030,18.42,82.61,523.8,0.08983,0.03766,0.02562,0.029230,0.1467,...,22.81,84.46,545.9,0.09701,0.04619,0.04833,0.05013,0.1987,0.06169,B
4,5,8.196,16.84,51.71,201.9,0.08600,0.05943,0.01588,0.005917,0.1769,...,21.96,57.26,242.2,0.12970,0.13570,0.06880,0.02564,0.3105,0.07409,B


C:\Users\domid\AppData\Local\Temp\ipykernel_5404\2763246943.py:41: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  dftemp = kagglehub.load_dataset(


'TEMPERATURA ESPAÑA'

,month,avg_temp,max_temp,min_temp,rain_days,rain_accum,avg_wind,place
0,Diciembre,10ºC,15ºC,6ºC,4 Días,23 mm,10,Murcia
1,Noviembre,13ºC,19ºC,9ºC,5 Días,27 mm,10,Murcia
2,Octubre,18ºC,24ºC,14ºC,5 Días,27 mm,8,Murcia
3,Septiembre,22ºC,29ºC,17ºC,3 Días,15 mm,8,Murcia
4,Agosto,25ºC,32ºC,19ºC,1 Días,3 mm,9,Murcia


# 2. Limpieza de Datasets y resultados sin incluir hiperparámetros
###  Preparación previa a la aplicación de los diferentes Algoritmos de Clasificación.
### Prueba del modelo sin incluir los hiperparámetros para comparar en futuros apartados.

## 2.1 TEMPERATURA ESPAÑA

### Info de las columnas:
| **Variable**         | **Descripción**                                     |
|----------------------|-----------------------------------------------------|
| **rain_days**        | Días de lluvia en un mes                           |
| **month**            | Mes del año                                        |
| **max_temp**         | Temperatura máxima durante el mes                  |
| **min_temp**         | Temperatura mínima durante el mes                  |
| **rain_accum**       | Acumulación de lluvia en el mes (en mm)            |
| **avg_wind**         | Velocidad media del viento durante el mes          |
| **avg_temp**         | Temperatura media durante el mes                   |
| **place**            | Lugar de la medición de los datos                  |

In [295]:
display(dftemp.head(5))

,month,avg_temp,max_temp,min_temp,rain_days,rain_accum,avg_wind,place
0,Diciembre,10ºC,15ºC,6ºC,4 Días,23 mm,10,Murcia
1,Noviembre,13ºC,19ºC,9ºC,5 Días,27 mm,10,Murcia
2,Octubre,18ºC,24ºC,14ºC,5 Días,27 mm,8,Murcia
3,Septiembre,22ºC,29ºC,17ºC,3 Días,15 mm,8,Murcia
4,Agosto,25ºC,32ºC,19ºC,1 Días,3 mm,9,Murcia


### Comprobamos los nulos

In [296]:
# Comprobar los valores nulos por columna
nulos_por_columna = dftemp.isnull().sum()

# Mostrar el resultado
print("Número de valores nulos por columna:")
print(nulos_por_columna)

Número de valores nulos por columna:
month         0
avg_temp      0
max_temp      0
min_temp      0
rain_days     0
rain_accum    0
avg_wind      0
place         0
dtype: int64


### No tenemos nulos, pero podemos descartar es avg_temp porque se podrá calcular con las demás, las otras serán importante para que nuestro modelo pueda predecir cuántos dias de lluvia habrá en el mes deseado

In [297]:
dftemp = dftemp.drop("avg_temp", axis=1)

### Antes del encoding, aunque no sea técnicamente obligatorio vamos a convertir a valores numéricos las columnas que en realidad son valores numericos (todas menos mes), aunque también se podria pasar a numérico pero la dejaremos así para que lo haga el Label Encoding mejor

In [298]:
# Convertir las columnas a formato numérico solo si no están ya en el formato adecuado

columns_to_convert = ["max_temp", "min_temp", "rain_days", "rain_accum"]

for col in columns_to_convert:
    try:
        # Si la columna ya es numérica, el código no hará nada
        if "temp" in col:  # Eliminar 'ºC' de las columnas de temperatura
            if dftemp[col].dtype != float:  # Verificamos si la columna no es de tipo float
                dftemp[col] = dftemp[col].str.replace("ºC", "").astype(float)
        elif "rain_days" in col:  # Eliminar ' Días' de la columna de días de lluvia
            if dftemp[col].dtype != int:  # Verificamos si la columna no es de tipo int
                dftemp[col] = dftemp[col].str.replace(" Días", "").astype(int)
        elif "rain_accum" in col:  # Eliminar ' mm' de la columna de acumulación de lluvia
            if dftemp[col].dtype != float:  # Verificamos si la columna no es de tipo float
                dftemp[col] = dftemp[col].str.replace(" mm", "").astype(float)
    except Exception as e:
        print(f"Error al convertir la columna {col}: {e}")

# Verificar el resultado
display(dftemp.head())
dftemp.info()

,month,max_temp,min_temp,rain_days,rain_accum,avg_wind,place
0,Diciembre,15.0,6.0,4,23.0,10,Murcia
1,Noviembre,19.0,9.0,5,27.0,10,Murcia
2,Octubre,24.0,14.0,5,27.0,8,Murcia
3,Septiembre,29.0,17.0,3,15.0,8,Murcia
4,Agosto,32.0,19.0,1,3.0,9,Murcia


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480 entries, 0 to 479
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   month       480 non-null    object 
 1   max_temp    480 non-null    float64
 2   min_temp    480 non-null    float64
 3   rain_days   480 non-null    int64  
 4   rain_accum  480 non-null    float64
 5   avg_wind    480 non-null    int64  
 6   place       480 non-null    object 
dtypes: float64(3), int64(2), object(2)
memory usage: 26.4+ KB


### Para pasar las columnas no numéricas, antes de empezar con las técnicas de escalado de características, vamos a usar One - hot encoding

In [299]:
# Crear una copia del DataFrame original para no modificar el original
dfesp_onehot = dftemp.copy()

# Filtrar las columnas categóricas
columnas_categoricas_temp = ["month", "place"]

# Instancia de OneHotEncoder
one_hot_encoder = OneHotEncoder(sparse_output=False, drop='first')

# Aplicar One-Hot Encoding a las columnas seleccionadas
columnas_encoded_temp = one_hot_encoder.fit_transform(dfesp_onehot[columnas_categoricas_temp])

# Convertir el resultado a un DataFrame
columnas_encoded_df_temp = pd.DataFrame(columnas_encoded_temp, columns=one_hot_encoder.get_feature_names_out(columnas_categoricas_temp))

# Ahora concatenamos las columnas numéricas (las que no son categóricas) al DataFrame resultante
columnas_numéricas = dfesp_onehot.drop(columns=columnas_categoricas_temp)
columnas_encoded_df_temp = pd.concat([columnas_encoded_df_temp, columnas_numéricas], axis=1)

# Mostrar el resultado
display(columnas_encoded_df_temp.head())

# Número de columnas antes y después del One-Hot Encoding
print("Número de columnas antes del One-Hot Encoding:", dfesp_onehot.shape[1])
print("Número de columnas después del One-Hot Encoding:", columnas_encoded_df_temp.shape[1])
#columnas_encoded_df_temp.info()

,month_Agosto,month_Diciembre,month_Enero,month_Febrero,month_Julio,month_Junio,month_Marzo,month_Mayo,month_Noviembre,month_Octubre,...,place_Valladolid,place_Zamora,place_Zaragoza,place_Álava,place_Ávila,max_temp,min_temp,rain_days,rain_accum,avg_wind
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,15.0,6.0,4,23.0,10
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,19.0,9.0,5,27.0,10
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,24.0,14.0,5,27.0,8
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,29.0,17.0,3,15.0,8
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,32.0,19.0,1,3.0,9


Número de columnas antes del One-Hot Encoding: 7
Número de columnas después del One-Hot Encoding: 55


### Ahora vamos a normalizar los datos para dejarlo todo listo para cuando apliquemos los diferentes algoritmos de clasificación, en este caso vamos a usar Normalización (Min-Max) porque es la opción más adecuada, ya que ajusta las escalas sin alterar la forma de los datos y mantiene la comparabilidad entre las variables. Además, no afecta a los valores de rain_days.

In [300]:
# Copia del dataframe para aplicar la normalización
df_esplimpio = columnas_encoded_df_temp.copy()

# Lista de las columnas numéricas para las que haremos la normalización
columnas_numericas = df_esplimpio.select_dtypes(include=['float64', 'int64']).columns

# Excluimos la columna objetivo (quality)
columnas_numericas = columnas_numericas.drop('rain_days')

# Aplicamos MinMaxScaler de sklearn para normalizar las columnas numéricas
min_max = MinMaxScaler()

# Aplicamos el ajuste y la transformación a las columnas numéricas
dfesp_limpio = pd.DataFrame(min_max.fit_transform(df_esplimpio[columnas_numericas]), columns=columnas_numericas)

# Copiamos la columna "quality" (sin normalizar) a la tabla final
dfesp_limpio['rain_days'] = df_esplimpio['rain_days']

display(dfesp_limpio.head(5))


,month_Agosto,month_Diciembre,month_Enero,month_Febrero,month_Julio,month_Junio,month_Marzo,month_Mayo,month_Noviembre,month_Octubre,...,place_Valladolid,place_Zamora,place_Zaragoza,place_Álava,place_Ávila,max_temp,min_temp,rain_accum,avg_wind,rain_days
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.28125,0.333333,0.106481,0.285714,4
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.40625,0.458333,0.125000,0.285714,5
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.56250,0.666667,0.125000,0.142857,5
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.71875,0.791667,0.069444,0.142857,3
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.81250,0.875000,0.013889,0.214286,1


## 2.2 PHISHING URLS

### Info de las columnas:
| **Variable**             | **Descripción**                                 | **Posibles Valores**                |
|--------------------------|-------------------------------------------------|-------------------------------------|
| **Index**                | Número de índice                                | Número de índice (0, 1, 2, ...)     |
| **UsingIP**              | Uso de IP (1: Usado, -1: No usado)              | { -1, 1 }                          |
| **LongURL**              | Longitud de URL (1: Larga, -1: Corta)           | { 1, 0, -1 }                       |
| **ShortURL**             | URL corta (1: Usada, -1: No usada)              | { 1, -1 }                          |
| **Symbol@**              | Uso del símbolo '@' en la URL                  | { 1, -1 }                          |
| **Redirecting//**        | URL redireccionada (1: Redirige, -1: No redirige)| { -1, 1 }                          |
| **PrefixSuffix-**        | Prefijo y sufijo en la URL (1: Tiene, -1: No tiene) | { -1, 1 }                          |
| **HTTPS**                | Uso de HTTPS (1: Usado, 0: No usado)            | { -1, 1, 0 }                       |
| **DomainRegLen**         | Longitud del registro de dominio                | { -1, 1 }                          |
| **Favicon**              | Presencia de favicon                            | { 1, -1 }                          |
| **NonStdPort**           | Puerto no estándar (1: Usado, -1: No usado)      | { 1, -1 }                          |
| **HTTPSDomainURL**       | HTTPS en dominio (1: Usado, -1: No usado)        | { -1, 1 }                          |
| **RequestURL**           | URL de solicitud                                | { 1, -1 }                          |
| **AnchorURL**            | URL de anclaje (1: Existe, -1: No existe, 0: No hay) | { -1, 0, 1 }                       |
| **LinksInScriptTags**    | Enlaces dentro de etiquetas `<script>`          | { 1, -1, 0 }                       |
| **ServerFormHandler**    | Manejo de formularios en servidor               | { -1, 1, 0 }                       |
| **InfoEmail**            | Información en el correo electrónico (1: Tiene, -1: No tiene) | { -1, 1 }                          |
| **AbnormalURL**          | URL anómala                                     | { -1, 1 }                          |
| **WebsiteForwarding**    | Redirección de sitio web (1: Redirige, 0: No redirige) | { 0, 1 }                           |
| **StatusBarCust**        | Personalización de la barra de estado           | { 1, -1 }                          |
| **DisableRightClick**    | Desactivar clic derecho (1: Desactivado, -1: No desactivado) | { 1, -1 }                          |
| **UsingPopupWindow**     | Uso de ventanas emergentes (1: Usado, -1: No usado) | { 1, -1 }                          |
| **IframeRedirection**    | Redirección mediante iframe (1: Usado, -1: No usado) | { 1, -1 }                          |
| **AgeofDomain**          | Edad del dominio (1: Viejo, -1: Nuevo)          | { -1, 1 }                          |
| **DNSRecording**         | Registro DNS (1: Tiene, -1: No tiene)           | { -1, 1 }                          |
| **WebsiteTraffic**       | Tráfico del sitio web (1: Alto, -1: Bajo, 0: No hay) | { -1, 0, 1 }                       |
| **PageRank**             | Rango de página de Google (1: Alto, -1: Bajo)   | { -1, 1 }                          |
| **GoogleIndex**          | Indexado en Google (1: Indexado, -1: No indexado) | { 1, -1 }                          |
| **LinksPointingToPage**  | Enlaces apuntando a la página (1: Varios, 0: Ninguno, -1: Pocos) | { 1, 0, -1 }                       |
| **StatsReport**          | Informe de estadísticas (1: Tiene, -1: No tiene) | { -1, 1 }                          |
| **class**                | Clase (1: Website normal, -1: Phishing)        | { -1, 1 }                          |


In [301]:
display(dfphishing.head(5))
dfphishing.info()

,Index,UsingIP,LongURL,ShortURL,Symbol@,Redirecting//,PrefixSuffix-,SubDomains,HTTPS,DomainRegLen,...,UsingPopupWindow,IframeRedirection,AgeofDomain,DNSRecording,WebsiteTraffic,PageRank,GoogleIndex,LinksPointingToPage,StatsReport,class
0,0,1,1,1,1,1,-1,0,1,-1,...,1,1,-1,-1,0,-1,1,1,1,-1
1,1,1,0,1,1,1,-1,-1,-1,-1,...,1,1,1,-1,1,-1,1,0,-1,-1
2,2,1,0,1,1,1,-1,-1,-1,1,...,1,1,-1,-1,1,-1,1,-1,1,-1
3,3,1,0,-1,1,1,-1,1,1,-1,...,-1,1,-1,-1,0,-1,1,1,1,1
4,4,-1,0,-1,1,-1,-1,1,1,-1,...,1,1,1,1,1,-1,1,-1,-1,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11054 entries, 0 to 11053
Data columns (total 32 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   Index                11054 non-null  int64
 1   UsingIP              11054 non-null  int64
 2   LongURL              11054 non-null  int64
 3   ShortURL             11054 non-null  int64
 4   Symbol@              11054 non-null  int64
 5   Redirecting//        11054 non-null  int64
 6   PrefixSuffix-        11054 non-null  int64
 7   SubDomains           11054 non-null  int64
 8   HTTPS                11054 non-null  int64
 9   DomainRegLen         11054 non-null  int64
 10  Favicon              11054 non-null  int64
 11  NonStdPort           11054 non-null  int64
 12  HTTPSDomainURL       11054 non-null  int64
 13  RequestURL           11054 non-null  int64
 14  AnchorURL            11054 non-null  int64
 15  LinksInScriptTags    11054 non-null  int64
 16  ServerFormHandler    1

### Como podemos ver, están todos los datos ya normalizados y no hay que hacer ninguna transformación, son todas de tipo int, lo unico que se me puede ocurrir es borrar los index, no lo vamos a necesitar

In [302]:
dfphishing_limpio = dfphishing.copy()
dfphishing_limpio = dfphishing_limpio.drop(['Index'],axis = 1)

## 2.3 CALIDAD DEL VINO

### Info de las columnas:
| **Variable**             | **Descripción**                                        | **Rango de Valores**            |
|--------------------------|--------------------------------------------------------|---------------------------------|
| **fixed acidity**         | Acidez fija en el vino                                  | 4.6 a 15.9                     |
| **volatile acidity**      | Acidez volátil en el vino                              | 0.12 a 1.58                    |
| **citric acid**           | Ácido cítrico en el vino                               | 0 a 1                          |
| **residual sugar**        | Azúcar residual en el vino                             | 0.9 a 15.5                     |
| **chlorides**             | Cloruros en el vino                                   | 0.01 a 0.61                    |
| **free sulfur dioxide**   | Dióxido de azufre libre en el vino                     | 1 a 68                         |
| **total sulfur dioxide**  | Dióxido de azufre total en el vino                     | 6 a 289                        |
| **density**               | Densidad del vino                                      | 0.99 a 1                       |
| **pH**                    | Nivel de pH del vino                                   | 2.74 a 4.01                    |
| **sulphates**             | Sulfatos en el vino                                    | 0.33 a 2                       |
| **alcohol**               | Contenido de alcohol en el vino                        | 8.4 a 14.9                     |
| **quality**               | Calidad del vino (1-10, 5 o menor es mala calidad)     | 3 a 8 (buena calidad si > 5)    |
| **Id**                    | Identificación del vino                                | 0 a 1597                       |

In [303]:
display(dfvino.head(5))
dfvino.info()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,Id
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,2
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,3
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,4


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1143 entries, 0 to 1142
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1143 non-null   float64
 1   volatile acidity      1143 non-null   float64
 2   citric acid           1143 non-null   float64
 3   residual sugar        1143 non-null   float64
 4   chlorides             1143 non-null   float64
 5   free sulfur dioxide   1143 non-null   float64
 6   total sulfur dioxide  1143 non-null   float64
 7   density               1143 non-null   float64
 8   pH                    1143 non-null   float64
 9   sulphates             1143 non-null   float64
 10  alcohol               1143 non-null   float64
 11  quality               1143 non-null   int64  
 12  Id                    1143 non-null   int64  
dtypes: float64(11), int64(2)
memory usage: 116.2 KB


### Teniendo en cuenta que los datos parecen que necesitan poca transformación, primero vamos a eliminar la columna id y transformar la columna quality que es nuestra variable objetiva a (si es 5 o menor es de mala calidad y si es mayor de 5 es buena calidad).
###  0 para mala calidad e 1 para buena calidad quedaría

In [304]:
dfvino = dfvino.drop(['Id'],axis = 1)
dfvino['quality'] = dfvino['quality'].apply(lambda x: 1 if x > 5 else 0)
display(dfvino.head(5))

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


### Vamos a escalar las variables categoricas usando StandardScaler porque las características, como la acidez o alcohol, tienen rangos diferentes. Estandarizarlas asegura que los algoritmos como SVM o kNN no se vean afectados por estas diferencias y aprendan de manera más efectiva. Además, es más robusto frente a valores atípicos que la normalización.

In [305]:
# Crear el objeto escalador
scaler = StandardScaler()

# Copiamos el dataframe para no modificar el original
dfvino_limpio = dfvino.copy()

# Estandarizar las columnas numéricas (exceptuando 'quality')
dfvino_limpio = dfvino.drop(columns=['quality'])
dfvino_limpio = scaler.fit_transform(dfvino_limpio)

# Convertir de nuevo a un DataFrame
dfvino_limpio = pd.DataFrame(dfvino_limpio, columns=dfvino.drop(columns=['quality']).columns)

# Incluir 'quality' nuevamente al dataframe
dfvino_limpio['quality'] = dfvino['quality']
display(dfvino_limpio.head(5))

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,-0.521580,0.939332,-1.365027,-0.466421,-0.231395,-0.450467,-0.363610,0.555854,1.270695,-0.573658,-0.963382,0
1,-0.292593,1.941813,-1.365027,0.050060,0.234247,0.915920,0.643477,0.036165,-0.708928,0.130881,-0.593601,0
2,-0.292593,1.273492,-1.161568,-0.171289,0.107253,-0.060071,0.246745,0.140103,-0.325775,-0.045254,-0.593601,0
3,1.653789,-1.399789,1.483400,-0.466421,-0.252560,0.135127,0.429852,0.659792,-0.964363,-0.456235,-0.593601,1
4,-0.521580,0.939332,-1.365027,-0.466421,-0.231395,-0.450467,-0.363610,0.555854,1.270695,-0.573658,-0.963382,0


## 2.4 DIAGNOSTICO DE CANCER DE MAMA

### Info de las columnas:
| Columnas                      | Min  | Max  | Descripción |
|------------------------------|------|------|-------------|
| x.radius_mean                 | 6.98 | 28.1 | Radio medio de las células |
| x.texture_mean                | 9.71 | 39.3 | Textura media de las células |
| x.perimeter_mean              | 43.8 | 189  | Perímetro medio de las células |
| x.area_mean                   | 144  | 2.5k | Área media de las células |
| x.smoothness_mean             | 0.05 | 0.16 | Suavidad media de las células |
| x.compactness_mean            | 0.02 | 0.35 | Compactación media de las células |
| x.concavity_mean              | 0    | 0.43 | Concavidad media de las células |
| x.concave_pts_mean            | 0    | 0.2  | Puntos cóncavos medios en las células |
| x.symmetry_mean               | 0.11 | 0.3  | Simetría media de las células |
| x.fractal_dim_mean            | 0.05 | 0.1  | Dimensión fractal media de las células |
| x.radius_se                   | 0.11 | 2.87 | Error estándar del radio |
| x.texture_se                  | 0.36 | 4.88 | Error estándar de la textura |
| x.perimeter_se                | 0.76 | 22   | Error estándar del perímetro |
| x.area_se                     | 6.8  | 542  | Error estándar del área |
| x.smoothness_se               | 0    | 0.03 | Error estándar de la suavidad |
| x.compactness_se              | 0    | 0.14 | Error estándar de la compactación |
| x.concavity_se                | 0    | 0.4  | Error estándar de la concavidad |
| x.concave_pts_se              | 0    | 0.05 | Error estándar de los puntos cóncavos |
| x.symmetry_se                 | 0.01 | 0.08 | Error estándar de la simetría |
| x.fractal_dim_se              | 0    | 0.03 | Error estándar de la dimensión fractal |
| x.radius_worst                | 7.93 | 36   | Radio peor (más grande) de las células |
| x.texture_worst               | 12   | 49.5 | Textura peor de las células |
| x.perimeter_worst             | 50.4 | 251  | Perímetro peor de las células |
| x.area_worst                  | 185  | 4.25k| Área peor de las células |
| x.smoothness_worst            | 0.07 | 0.22 | Suavidad peor de las células |
| x.compactness_worst           | 0.03 | 1.06 | Compactación peor de las células |
| x.concavity_worst             | 0    | 1.25 | Concavidad peor de las células |
| x.concave_pts_worst           | 0    | 0.29 | Puntos cóncavos peores en las células |
| x.symmetry_worst              | 0.16 | 0.66 | Simetría peor de las células |
| x.fractal_dim_worst           | 0.06 | 0.21 | Dimensión fractal peor de las células |
| y (variable objetiva)                 | B (63%) | M (37%) | Diagnóstico: B = benigno, M = maligno |



In [306]:
display(dfcancer.head(5))
dfcancer.info()

,Unnamed: 0,x.radius_mean,x.texture_mean,x.perimeter_mean,x.area_mean,x.smoothness_mean,x.compactness_mean,x.concavity_mean,x.concave_pts_mean,x.symmetry_mean,...,x.texture_worst,x.perimeter_worst,x.area_worst,x.smoothness_worst,x.compactness_worst,x.concavity_worst,x.concave_pts_worst,x.symmetry_worst,x.fractal_dim_worst,y
0,1,13.540,14.36,87.46,566.3,0.09779,0.08129,0.06664,0.047810,0.1885,...,19.26,99.70,711.2,0.14400,0.17730,0.23900,0.12880,0.2977,0.07259,B
1,2,13.080,15.71,85.63,520.0,0.10750,0.12700,0.04568,0.031100,0.1967,...,20.49,96.09,630.5,0.13120,0.27760,0.18900,0.07283,0.3184,0.08183,B
2,3,9.504,12.44,60.34,273.9,0.10240,0.06492,0.02956,0.020760,0.1815,...,15.66,65.13,314.9,0.13240,0.11480,0.08867,0.06227,0.2450,0.07773,B
3,4,13.030,18.42,82.61,523.8,0.08983,0.03766,0.02562,0.029230,0.1467,...,22.81,84.46,545.9,0.09701,0.04619,0.04833,0.05013,0.1987,0.06169,B
4,5,8.196,16.84,51.71,201.9,0.08600,0.05943,0.01588,0.005917,0.1769,...,21.96,57.26,242.2,0.12970,0.13570,0.06880,0.02564,0.3105,0.07409,B


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 32 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           569 non-null    int64  
 1   x.radius_mean        569 non-null    float64
 2   x.texture_mean       569 non-null    float64
 3   x.perimeter_mean     569 non-null    float64
 4   x.area_mean          569 non-null    float64
 5   x.smoothness_mean    569 non-null    float64
 6   x.compactness_mean   569 non-null    float64
 7   x.concavity_mean     569 non-null    float64
 8   x.concave_pts_mean   569 non-null    float64
 9   x.symmetry_mean      569 non-null    float64
 10  x.fractal_dim_mean   569 non-null    float64
 11  x.radius_se          569 non-null    float64
 12  x.texture_se         569 non-null    float64
 13  x.perimeter_se       569 non-null    float64
 14  x.area_se            569 non-null    float64
 15  x.smoothness_se      569 non-null    flo

### Primero para tenerlo todo numérico, vamos a pasar la variable objetivo a numérico cuando el diagnóstico es B será 0 y M = 1

In [307]:
### Copiamos el dataframe para no modificar el original
dfcancer_limpio = dfcancer.copy()
dfcancer_limpio['y'] = dfcancer_limpio['y'].map({'B': 0, 'M': 1})
display(dfcancer_limpio['y'].head(5))

0    0
1    0
2    0
3    0
4    0
Name: y, dtype: int64

### Comprobamos que no hayan valores nulos

In [308]:
# Comprobar los valores nulos por columna
nulos_cancer_xcolumna = dfcancer_limpio.isnull().sum()

# Mostrar el resultado
print("Número de valores nulos por columna:")
print(nulos_cancer_xcolumna)

Número de valores nulos por columna:
Unnamed: 0             0
x.radius_mean          0
x.texture_mean         0
x.perimeter_mean       0
x.area_mean            0
x.smoothness_mean      0
x.compactness_mean     0
x.concavity_mean       0
x.concave_pts_mean     0
x.symmetry_mean        0
x.fractal_dim_mean     0
x.radius_se            0
x.texture_se           0
x.perimeter_se         0
x.area_se              0
x.smoothness_se        0
x.compactness_se       0
x.concavity_se         0
x.concave_pts_se       0
x.symmetry_se          0
x.fractal_dim_se       0
x.radius_worst         0
x.texture_worst        0
x.perimeter_worst      0
x.area_worst           0
x.smoothness_worst     0
x.compactness_worst    0
x.concavity_worst      0
x.concave_pts_worst    0
x.symmetry_worst       0
x.fractal_dim_worst    0
y                      0
dtype: int64


### Parece que no hay nulos, vamos a borrar la primera columna que nos aparece como Unnamed: 0 que viendola arriba en los head mostrados, parece ser como una especie de índice

In [309]:
### Borramos Unnamed: 0
dfcancer_limpio = dfcancer_limpio.drop("Unnamed: 0", axis=1)

### Ahora vamos con la normalización de los datos utilizando StandardScaler porque escala las características para que tengan una media de 0 y una desviación estándar de 1, lo que mejora la convergencia de algoritmos como la regresión logística o máquinas de soporte vectorial, que son sensibles a la escala de los datos. Esto garantiza que todas las variables contribuyan de manera equitativa al modelo, porque siendo sincero no sabría a que variable tendría que centrarme más para darle más peso.

In [310]:
# Escalar todas las columnas numéricas, excepto la columna 'y'
scaler = StandardScaler()

# Seleccionar las características (eliminar 'y' antes de aplicar el escalado)
X = dfcancer_limpio.drop('y', axis=1)

# Aplicar el escalado
X_scaled = scaler.fit_transform(X)

# Reemplazar el DataFrame original con los valores escalados
dfcancer_limpio_scaled = pd.DataFrame(X_scaled, columns=X.columns)

# Volver a añadir la columna 'y' al DataFrame escalado
dfcancer_limpio_scaled['y'] = dfcancer_limpio['y'].reset_index(drop=True)

# Ahora dfcancer_limpio_scaled contiene los datos escalados
dfcancer_limpio = dfcancer_limpio_scaled
display(dfcancer_limpio.head(5))

,x.radius_mean,x.texture_mean,x.perimeter_mean,x.area_mean,x.smoothness_mean,x.compactness_mean,x.concavity_mean,x.concave_pts_mean,x.symmetry_mean,x.fractal_dim_mean,...,x.texture_worst,x.perimeter_worst,x.area_worst,x.smoothness_worst,x.compactness_worst,x.concavity_worst,x.concave_pts_worst,x.symmetry_worst,x.fractal_dim_worst,y
0,-0.166799,-1.147162,-0.185728,-0.251957,0.101747,-0.436850,-0.278210,-0.028609,0.267911,-0.728310,...,-1.045005,-0.225217,-0.297761,0.509873,-0.489605,-0.159223,0.216123,0.123347,-0.629292,0
1,-0.297446,-0.833008,-0.261106,-0.383638,0.792763,0.429422,-0.541362,-0.459627,0.567289,0.753087,...,-0.844707,-0.332744,-0.439624,-0.051226,0.148443,-0.399099,-0.636110,0.458227,-0.117250,0
2,-1.313080,-1.593959,-1.302806,-1.083572,0.429819,-0.747086,-0.743748,-0.726337,0.012345,0.886341,...,-1.631243,-1.254913,-0.994422,0.001377,-0.887193,-0.880434,-0.796903,-0.729224,-0.344455,0
3,-0.311646,-0.202373,-0.385500,-0.372831,-0.464730,-1.263703,-0.793214,-0.507861,-1.258183,-0.590802,...,-0.466909,-0.679153,-0.588344,-1.549975,-1.323648,-1.073966,-0.981753,-1.478256,-1.233324,0
4,-1.684571,-0.570050,-1.658278,-1.288347,-0.737294,-0.851130,-0.915500,-1.109197,-0.155598,0.316465,...,-0.605327,-1.489328,-1.122222,-0.116980,-0.754239,-0.975761,-1.354653,0.330422,-0.546168,0


# 3. Aplicación de los diferentes algoritmos de clasificación

***

## `3.1 Support Vector Machine (SVM)`

### El algoritmo SVM busca encontrar un hiperplano en un espacio multidimensional que separe las clases de manera óptima. El objetivo es maximizar el margen entre las clases, es decir, la distancia entre el hiperplano y las muestras más cercanas de cada clase (llamadas vectores de soporte). Es muy útil para problemas de clasificación binaria, especialmente cuando las clases no son linealmente separables, gracias a los "kernels" que permiten transformar los datos a un espacio de mayor dimensión.
### La fórmula general para **SVM** es:

$$
f(x) = \text{sign} \left( \sum_{i=1}^{n} \alpha_i y_i K(x_i, x) + b \right)
$$

### Donde:
- αᵢ (Coeficientes de Lagrange): Son multiplicadores que determinan la importancia de cada punto de entrenamiento en el modelo SVM.
- yᵢ (Etiquetas de clase): Representan la clase o categoría de cada punto de datos (usualmente -1 o +1).
- K(xᵢ, x) (Kernel): Es una función que mide la similitud entre dos puntos de datos, permitiendo transformar los datos en un espacio de mayor dimensión.
- b (Sesgo): Ajusta la posición del hiperplano de separación para mejorar la clasificación entre las clases.


### Método para aplicar a cada dataset:
### Como tenemos varios dataset que algunos abordan problemas de regresión y otros de clasificación, entonces he optado por separar dichos conceptos a la hora de elegir el modelo y métricas en este caso:
* Para problemas de regresión: SVR() y métricas (MAE, MSE y R2 Score)
* Para problemas de clasificación: SVC() y métricas (Exactitud, F1 Score, Sensibilidad y Precisión)

In [311]:
# Nuevas métricas de regresión
def metricas_regresion_adicionales(y_true, y_pred):
    # RMSE
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    
    # Explained Variance Score
    evs = explained_variance_score(y_true, y_pred)
    
    return rmse, evs

# Nuevas métricas de clasificación
def metricas_clasificacion_adicionales(y_true, y_pred, y_prob=None):
    # AUC-ROC
    auc_roc = roc_auc_score(y_true, y_prob) if y_prob is not None else None
    
    # Precision-Recall AUC
    pr_auc = average_precision_score(y_true, y_prob) if y_prob is not None else None
    
    return auc_roc, pr_auc

# Función para mostrar métricas de regresión
def mostrarMetricasRegresion(modelo, metrica, valor_metrica_entrenamiento, valor_metrica_testeo):
    print("{} : {} en los datos de entrenamiento: {:.3f}".format(modelo, metrica, valor_metrica_entrenamiento))
    print("{} : {} en los datos de testeo: {:.3f}".format(modelo, metrica, valor_metrica_testeo))
    print()

# Función para mostrar métricas de clasificación
def mostrarMetricasClasificacion(modelo, metrica, valor_metrica_entrenamiento, valor_metrica_testeo):
    print("{} : {} en los datos de entrenamiento: {:.3f}".format(modelo, metrica, valor_metrica_entrenamiento))
    print("{} : {} en los datos de testeo: {:.3f}".format(modelo, metrica, valor_metrica_testeo))
    print()

# Función para entrenar y evaluar un modelo SVM
def aplicar_svm(df, target_column, tipo_modelo='regresion'):
    # Separar las características (X) y la variable objetivo (y)
    X = df.drop(columns=[target_column])
    y = df[target_column]
    
    # Dividir el dataset en entrenamiento (80%) y prueba (20%)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Seleccionar el modelo SVM dependiendo del tipo (regresión o clasificación)
    if tipo_modelo == 'regresion':
        model = SVR()
    elif tipo_modelo == 'clasificacion':
        model = SVC(probability=True)

    # Entrenar el modelo con los datos de entrenamiento
    model.fit(X_train, y_train)
    
    # Hacer predicciones sobre los conjuntos de entrenamiento y prueba
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluar el modelo con las métricas correspondientes
    if tipo_modelo == 'regresion':
        # Métricas existentes
        mae_train = mean_absolute_error(y_train, y_train_pred)
        mae_test = mean_absolute_error(y_test, y_test_pred)
        mostrarMetricasRegresion("SVM", "MAE", mae_train, mae_test)
        
        mse_train = mean_squared_error(y_train, y_train_pred)
        mse_test = mean_squared_error(y_test, y_test_pred)
        mostrarMetricasRegresion("SVM", "MSE", mse_train, mse_test)
        
        r2_train = r2_score(y_train, y_train_pred)
        r2_test = r2_score(y_test, y_test_pred)
        mostrarMetricasRegresion("SVM", "R2 Score", r2_train, r2_test)
        
        # Nuevas métricas
        rmse_train, evs_train = metricas_regresion_adicionales(y_train, y_train_pred)
        rmse_test, evs_test = metricas_regresion_adicionales(y_test, y_test_pred)
        mostrarMetricasRegresion("SVM", "RMSE", rmse_train, rmse_test)
        mostrarMetricasRegresion("SVM", "Explained Variance", evs_train, evs_test)
    
    elif tipo_modelo == 'clasificacion':
        # Métricas existentes
        exactitud_train = metrics.accuracy_score(y_train, y_train_pred)
        exactitud_test = metrics.accuracy_score(y_test, y_test_pred)
        mostrarMetricasClasificacion("SVM", "Exactitud", exactitud_train, exactitud_test)
        
        f1_train = metrics.f1_score(y_train, y_train_pred, average='binary')  # Cambiar a 'micro' o 'macro' si es multiclase
        f1_test = metrics.f1_score(y_test, y_test_pred, average='binary')
        mostrarMetricasClasificacion("SVM", "F1 Score", f1_train, f1_test)
        
        sensibilidad_train = metrics.recall_score(y_train, y_train_pred)
        sensibilidad_test = metrics.recall_score(y_test, y_test_pred)
        mostrarMetricasClasificacion("SVM", "Sensibilidad", sensibilidad_train, sensibilidad_test)
        
        precision_train = metrics.precision_score(y_train, y_train_pred)
        precision_test = metrics.precision_score(y_test, y_test_pred)
        mostrarMetricasClasificacion("SVM", "Precisión", precision_train, precision_test)
        
        # Probabilidades para AUC-ROC y Precision-Recall AUC
        y_train_prob = model.predict_proba(X_train)[:, 1]
        y_test_prob = model.predict_proba(X_test)[:, 1]
        
        # Nuevas métricas
        auc_roc_train, pr_auc_train = metricas_clasificacion_adicionales(y_train, y_train_pred, y_train_prob)
        auc_roc_test, pr_auc_test = metricas_clasificacion_adicionales(y_test, y_test_pred, y_test_prob)
        mostrarMetricasClasificacion("SVM", "AUC-ROC", auc_roc_train, auc_roc_test)
        mostrarMetricasClasificacion("SVM", "Precision-Recall AUC", pr_auc_train, pr_auc_test)
    
    return model


### Métricas que hemos añadido aparte:
* Para regresión: RMSE (Raíz del Error Cuadrático Medio) y Explained Variance Score (Puntaje de Varianza Explicada)
* Para clasificación: AUC-ROC (Área Bajo la Curva ROC) y Curva Precision-Recall

### RMSE (Raíz del Error Cuadrático Medio)
### Es la raíz cuadrada del error cuadrático medio (MSE). Esta métrica es útil para medir cuánto se alejan en promedio las predicciones del modelo de los valores reales. Se expresa en las mismas unidades que la variable de salida, lo que la hace más interpretable que el MSE.
### Fórmula:
$$
RMSE = \sqrt{MSE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2}
$$

Donde:
- y<sub>i</sub> son los valores reales,
- y<sup>̂</sup><sub>i</sub> son los valores predichos,
- n es el número total de datos.
​


***

### Explained Variance Score
### Mide qué tan bien las predicciones del modelo capturan la variabilidad de los datos reales(cuán variados o dispersos son los valores de la variable que estamos tratando de predecir). Un valor cercano a 1 indica que el modelo captura casi toda la variabilidad de los datos, mientras que un valor cercano a 0 indica que el modelo no tiene capacidad explicativa.
### La fórmula del **Explained Variance Score** es:
$$
\text{Explained Variance} = 1 - \frac{Var(y_{\text{true}} - y_{\text{pred}})}{Var(y_{\text{true}})}
$$

Donde:
- y<sub>true</sub> son los valores reales,
- y<sub>pred</sub> son los valores predichos,
- Var denota la varianza de los valores.

***

### **AUC-ROC (Área Bajo la Curva ROC)**
### Es una medida que evalúa la capacidad del modelo para distinguir entre las clases. Un valor de AUC cercano a 1 indica un buen rendimiento del modelo, mientras que un valor cercano a 0.5 sugiere que el modelo no tiene capacidad predictiva.
### La fórmula del **AUC-ROC** es:
$$
AUC-ROC = \int_0^1 TPR(FPR) \, dFPR
$$

Donde:
- TPR (True Positive Rate) es la sensibilidad o **recall**.
- FPR (False Positive Rate) es la tasa de falsos positivos.

***

###  **Precision-Recall AUC (Área Bajo la Curva Precision-Recall)**
###  Muestra la relación entre la precisión y el recall a diferentes umbrales de decisión. Es especialmente útil cuando las clases son desbalanceadas, ya que se enfoca en las predicciones de la clase positiva.
### La **Precision-Recall AUC** mide el área bajo la curva que traza la **Precisión** frente al **Recall** en diferentes umbrales de decisión:
$$
AUC_{\text{PR}} = \int_0^1 Precision(Recall) \, dRecall
$$

Donde:
- Precision es la proporción de verdaderos positivos sobre el total de positivos predichos.
- Recall es la proporción de verdaderos positivos sobre el total de positivos reales.

***

## `3.1.1 SVM Aplicado a TEMPERATURA ESPAÑA` (sin hiperparámetros)

In [312]:
# Ejemplo de uso para un modelo de regresión:
svm_modeltemp = aplicar_svm(dfesp_limpio, 'rain_days', tipo_modelo='regresion')

SVM : MAE en los datos de entrenamiento: 0.584
SVM : MAE en los datos de testeo: 0.980

SVM : MSE en los datos de entrenamiento: 0.851
SVM : MSE en los datos de testeo: 1.827

SVM : R2 Score en los datos de entrenamiento: 0.911
SVM : R2 Score en los datos de testeo: 0.800

SVM : RMSE en los datos de entrenamiento: 0.922
SVM : RMSE en los datos de testeo: 1.351

SVM : Explained Variance en los datos de entrenamiento: 0.912
SVM : Explained Variance en los datos de testeo: 0.802



## `3.1.2 SVM Aplicado a PHISHING URLS` (sin hiperparámetros)

In [313]:
# Aplicar SVM al dataset dfphishing_limpio
svm_modelphishing = aplicar_svm(dfphishing_limpio, 'class', tipo_modelo='clasificacion')

SVM : Exactitud en los datos de entrenamiento: 0.953
SVM : Exactitud en los datos de testeo: 0.951

SVM : F1 Score en los datos de entrenamiento: 0.958
SVM : F1 Score en los datos de testeo: 0.957

SVM : Sensibilidad en los datos de entrenamiento: 0.969
SVM : Sensibilidad en los datos de testeo: 0.973

SVM : Precisión en los datos de entrenamiento: 0.947
SVM : Precisión en los datos de testeo: 0.941

SVM : AUC-ROC en los datos de entrenamiento: 0.991
SVM : AUC-ROC en los datos de testeo: 0.989

SVM : Precision-Recall AUC en los datos de entrenamiento: 0.993
SVM : Precision-Recall AUC en los datos de testeo: 0.991



## `3.1.3 SVM Aplicado a CALIDAD DEL VINO` (sin hiperparámetros)

In [314]:
# Aplicar SVM al dataset dfvino_limpio
svm_modelvino = aplicar_svm(dfvino_limpio, 'quality', tipo_modelo='clasificacion')

SVM : Exactitud en los datos de entrenamiento: 0.802
SVM : Exactitud en los datos de testeo: 0.764

SVM : F1 Score en los datos de entrenamiento: 0.814
SVM : F1 Score en los datos de testeo: 0.791

SVM : Sensibilidad en los datos de entrenamiento: 0.804
SVM : Sensibilidad en los datos de testeo: 0.803

SVM : Precisión en los datos de entrenamiento: 0.825
SVM : Precisión en los datos de testeo: 0.779

SVM : AUC-ROC en los datos de entrenamiento: 0.878
SVM : AUC-ROC en los datos de testeo: 0.874

SVM : Precision-Recall AUC en los datos de entrenamiento: 0.892
SVM : Precision-Recall AUC en los datos de testeo: 0.898



## `3.1.4 SVM Aplicado a DIAGNOSTICO DE CANCER DE MAMA` (sin hiperparámetros)

In [315]:
# Aplicar SVM al dataset dfcancer_limpio
svm_modelcancer = aplicar_svm(dfcancer_limpio, 'y', tipo_modelo='clasificacion')

SVM : Exactitud en los datos de entrenamiento: 0.987
SVM : Exactitud en los datos de testeo: 0.956

SVM : F1 Score en los datos de entrenamiento: 0.982
SVM : F1 Score en los datos de testeo: 0.941

SVM : Sensibilidad en los datos de entrenamiento: 0.970
SVM : Sensibilidad en los datos de testeo: 0.930

SVM : Precisión en los datos de entrenamiento: 0.994
SVM : Precisión en los datos de testeo: 0.952

SVM : AUC-ROC en los datos de entrenamiento: 0.998
SVM : AUC-ROC en los datos de testeo: 0.997

SVM : Precision-Recall AUC en los datos de entrenamiento: 0.997
SVM : Precision-Recall AUC en los datos de testeo: 0.996



***

## `3.2 Regresión Logística`

### La regresión logística es un modelo estadístico que se utiliza para la clasificación binaria. Calcula la probabilidad de que una instancia pertenezca a una clase usando la función sigmoide, que mapea cualquier valor real a un valor entre 0 y 1. Luego, se asigna la clase con base en un umbral (usualmente 0.5). Aunque el nombre tiene "regresión", es un algoritmo de clasificación.
### La fórmula de **Regresión Logística** es:

$$
p(y=1|x) = \frac{1}{1 + e^{-(w^T x + b)}}
$$

### Donde:
- **w** (Pesos): Son los parámetros del modelo que se ajustan durante el entrenamiento para maximizar la probabilidad de la clasificación correcta.
- **x** (Características): Representan las variables de entrada del modelo.
- **b** (Sesgo): Es el valor adicional que permite ajustar la posición de la curva sigmoide.
- **p(y=1|x)** (Probabilidad de que la clase sea 1): Es la probabilidad de que el punto de datos pertenezca a la clase positiva.

### Aqui vamos a tener un problema, este método de regresión logística no tendría sentido aplicarlo en problemas de regresión, aunque su nombre puede ser confuso, la regresión logística es un modelo de clasificación, no de regresión. No debe usarse en problemas donde la variable objetivo es continua como en mi caso con rain_days del dataset TEMPERATURAS ESPAÑA.
### Para los demás algoritmos y datasets no vamos a tener este problema.
### Vamos a tener entonces en esta función:
* LogisticRegression() y métricas (Exactitud, F1 Score, Sensibilidad y Precisión) y añadidas (AUC-ROC y Precision-Recall AUC)

In [316]:
# Función para entrenar y evaluar un modelo de Regresión Logística
def aplicar_regresion_logistica(df, target_column, tipo_modelo='clasificacion'):
    # Separar las características (X) y la variable objetivo (y)
    X = df.drop(columns=[target_column])
    y = df[target_column]
    
    # Dividir el dataset en entrenamiento (80%) y prueba (20%)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Seleccionar el modelo de Regresión Logística
    model = LogisticRegression()
    
    # Entrenar el modelo con los datos de entrenamiento
    model.fit(X_train, y_train)
    
    # Hacer predicciones sobre los conjuntos de entrenamiento y prueba
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluar el modelo con las métricas correspondientes
    if tipo_modelo == 'clasificacion':
        # Métricas existentes
        exactitud_train = metrics.accuracy_score(y_train, y_train_pred)
        exactitud_test = metrics.accuracy_score(y_test, y_test_pred)
        mostrarMetricasClasificacion("Regresión Logística", "Exactitud", exactitud_train, exactitud_test)
        
        f1_train = metrics.f1_score(y_train, y_train_pred, average='binary')  # Cambiar a 'micro' o 'macro' si es multiclase
        f1_test = metrics.f1_score(y_test, y_test_pred, average='binary')
        mostrarMetricasClasificacion("Regresión Logística", "F1 Score", f1_train, f1_test)
        
        sensibilidad_train = metrics.recall_score(y_train, y_train_pred)
        sensibilidad_test = metrics.recall_score(y_test, y_test_pred)
        mostrarMetricasClasificacion("Regresión Logística", "Sensibilidad", sensibilidad_train, sensibilidad_test)
        
        precision_train = metrics.precision_score(y_train, y_train_pred)
        precision_test = metrics.precision_score(y_test, y_test_pred)
        mostrarMetricasClasificacion("Regresión Logística", "Precisión", precision_train, precision_test)
        
        # Probabilidades para AUC-ROC y Precision-Recall AUC
        y_train_prob = model.predict_proba(X_train)[:, 1]
        y_test_prob = model.predict_proba(X_test)[:, 1]
        
        # Nuevas métricas
        auc_roc_train, pr_auc_train = metricas_clasificacion_adicionales(y_train, y_train_pred, y_train_prob)
        auc_roc_test, pr_auc_test = metricas_clasificacion_adicionales(y_test, y_test_pred, y_test_prob)
        mostrarMetricasClasificacion("Regresión Logística", "AUC-ROC", auc_roc_train, auc_roc_test)
        mostrarMetricasClasificacion("Regresión Logística", "Precision-Recall AUC", pr_auc_train, pr_auc_test)
    
    elif tipo_modelo != 'clasificacion':
        print("Tipo de modelo no válido. Debe ser 'clasificacion'.")
    
    return model

## `3.2.1 Regresión logística Aplicado a TEMPERATURA ESPAÑA` (sin hiperparámetros)

In [317]:
# Explicado arriba, no se puede usar para este dataset.

## `3.2.2 Regresión logística Aplicado a PHISHING URLS` (sin hiperparámetros)

In [318]:
# Aplicar Regresión logística al dataset dfphishing_limpio
RL_modelphishing = aplicar_regresion_logistica(dfphishing_limpio, 'class', tipo_modelo='clasificacion')

Regresión Logística : Exactitud en los datos de entrenamiento: 0.927
Regresión Logística : Exactitud en los datos de testeo: 0.934

Regresión Logística : F1 Score en los datos de entrenamiento: 0.935
Regresión Logística : F1 Score en los datos de testeo: 0.941

Regresión Logística : Sensibilidad en los datos de entrenamiento: 0.943
Regresión Logística : Sensibilidad en los datos de testeo: 0.953

Regresión Logística : Precisión en los datos de entrenamiento: 0.927
Regresión Logística : Precisión en los datos de testeo: 0.930

Regresión Logística : AUC-ROC en los datos de entrenamiento: 0.979
Regresión Logística : AUC-ROC en los datos de testeo: 0.980

Regresión Logística : Precision-Recall AUC en los datos de entrenamiento: 0.983
Regresión Logística : Precision-Recall AUC en los datos de testeo: 0.984



## `3.2.3 Regresión logística Aplicado a CALIDAD DEL VINO` (sin hiperparámetros)

In [319]:
# Aplicar Regresión logística al dataset dfvino_limpio
RL_modelvino = aplicar_regresion_logistica(dfvino_limpio, 'quality', tipo_modelo='clasificacion')

Regresión Logística : Exactitud en los datos de entrenamiento: 0.758
Regresión Logística : Exactitud en los datos de testeo: 0.769

Regresión Logística : F1 Score en los datos de entrenamiento: 0.774
Regresión Logística : F1 Score en los datos de testeo: 0.791

Regresión Logística : Sensibilidad en los datos de entrenamiento: 0.767
Regresión Logística : Sensibilidad en los datos de testeo: 0.787

Regresión Logística : Precisión en los datos de entrenamiento: 0.781
Regresión Logística : Precisión en los datos de testeo: 0.794

Regresión Logística : AUC-ROC en los datos de entrenamiento: 0.829
Regresión Logística : AUC-ROC en los datos de testeo: 0.822

Regresión Logística : Precision-Recall AUC en los datos de entrenamiento: 0.853
Regresión Logística : Precision-Recall AUC en los datos de testeo: 0.835



## `3.2.4 Regresión logística Aplicado a DIAGNOSTICO DE CANCER DE MAMA` (sin hiperparámetros)

In [320]:
# Aplicar Regresión logística al dataset dfcancer_limpio
RL_modelcancer = aplicar_regresion_logistica(dfcancer_limpio, 'y', tipo_modelo='clasificacion')

Regresión Logística : Exactitud en los datos de entrenamiento: 0.991
Regresión Logística : Exactitud en los datos de testeo: 0.965

Regresión Logística : F1 Score en los datos de entrenamiento: 0.988
Regresión Logística : F1 Score en los datos de testeo: 0.953

Regresión Logística : Sensibilidad en los datos de entrenamiento: 0.976
Regresión Logística : Sensibilidad en los datos de testeo: 0.953

Regresión Logística : Precisión en los datos de entrenamiento: 1.000
Regresión Logística : Precisión en los datos de testeo: 0.953

Regresión Logística : AUC-ROC en los datos de entrenamiento: 0.998
Regresión Logística : AUC-ROC en los datos de testeo: 0.995

Regresión Logística : Precision-Recall AUC en los datos de entrenamiento: 0.997
Regresión Logística : Precision-Recall AUC en los datos de testeo: 0.993



***

# `3.3 Árboles de Decisión`

### Los árboles de decisión son un modelo de clasificación en el que los datos son segmentados en función de características. Cada nodo en el árbol representa una característica, y cada rama es un valor posible de esa característica. El proceso continúa hasta llegar a una hoja que representa la predicción final. Son fáciles de interpretar y eficaces para datos categóricos y continuos.
### La fórmula general para un **Árbol de Decisión** es:

$$
y = \text{Decisión}(x) = \text{Regla de Clasificación}
$$

### Donde:
- **x** (Características): Son las variables de entrada del modelo.
- **Regla de Clasificación**: Se toma la decisión sobre la clase en función de los umbrales que definen las divisiones del árbol.

### Vamos a tener entonces en esta función para aplicar a cada dataset:
### para clasificación:
* DecisionTreeClassifier()  y métricas (Exactitud, F1 Score, Sensibilidad y Precisión) y añadidas (AUC-ROC y Precision-Recall AUC)
### para regresión:
* DecisionTreeRegressor() y métricas (MAE, MSE, R2 Score) y añadidas (RMSE y Explained Variance)

In [321]:
# Función para entrenar y evaluar un modelo de Árbol de Decisión
def aplicar_arbol_decision(df, target_column, tipo_modelo='regresion'):
    # Separar las características (X) y la variable objetivo (y)
    X = df.drop(columns=[target_column])
    y = df[target_column]
    
    # Dividir el dataset en entrenamiento (80%) y prueba (20%)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Seleccionar el modelo dependiendo del tipo (regresión o clasificación)
    if tipo_modelo == 'regresion':
        model = DecisionTreeRegressor()
    elif tipo_modelo == 'clasificacion':
        model = DecisionTreeClassifier()

    # Entrenar el modelo con los datos de entrenamiento
    model.fit(X_train, y_train)
    
    # Hacer predicciones sobre los conjuntos de entrenamiento y prueba
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluar el modelo con las métricas correspondientes
    if tipo_modelo == 'regresion':
        # Métricas existentes
        mae_train = mean_absolute_error(y_train, y_train_pred)
        mae_test = mean_absolute_error(y_test, y_test_pred)
        mostrarMetricasRegresion("Árbol de Decisión", "MAE", mae_train, mae_test)
        
        mse_train = mean_squared_error(y_train, y_train_pred)
        mse_test = mean_squared_error(y_test, y_test_pred)
        mostrarMetricasRegresion("Árbol de Decisión", "MSE", mse_train, mse_test)
        
        r2_train = r2_score(y_train, y_train_pred)
        r2_test = r2_score(y_test, y_test_pred)
        mostrarMetricasRegresion("Árbol de Decisión", "R2 Score", r2_train, r2_test)
        
        # Nuevas métricas
        rmse_train, evs_train = metricas_regresion_adicionales(y_train, y_train_pred)
        rmse_test, evs_test = metricas_regresion_adicionales(y_test, y_test_pred)
        mostrarMetricasRegresion("Árbol de Decisión", "RMSE", rmse_train, rmse_test)
        mostrarMetricasRegresion("Árbol de Decisión", "Explained Variance", evs_train, evs_test)
    
    elif tipo_modelo == 'clasificacion':
        # Métricas existentes
        exactitud_train = metrics.accuracy_score(y_train, y_train_pred)
        exactitud_test = metrics.accuracy_score(y_test, y_test_pred)
        mostrarMetricasClasificacion("Árbol de Decisión", "Exactitud", exactitud_train, exactitud_test)
        
        f1_train = metrics.f1_score(y_train, y_train_pred, average='binary')  # Cambiar a 'micro' o 'macro' si es multiclase
        f1_test = metrics.f1_score(y_test, y_test_pred, average='binary')
        mostrarMetricasClasificacion("Árbol de Decisión", "F1 Score", f1_train, f1_test)
        
        sensibilidad_train = metrics.recall_score(y_train, y_train_pred)
        sensibilidad_test = metrics.recall_score(y_test, y_test_pred)
        mostrarMetricasClasificacion("Árbol de Decisión", "Sensibilidad", sensibilidad_train, sensibilidad_test)
        
        precision_train = metrics.precision_score(y_train, y_train_pred)
        precision_test = metrics.precision_score(y_test, y_test_pred)
        mostrarMetricasClasificacion("Árbol de Decisión", "Precisión", precision_train, precision_test)
        
        # Probabilidades para AUC-ROC y Precision-Recall AUC
        y_train_prob = model.predict_proba(X_train)[:, 1] if hasattr(model, 'predict_proba') else None
        y_test_prob = model.predict_proba(X_test)[:, 1] if hasattr(model, 'predict_proba') else None
        
        # Nuevas métricas
        auc_roc_train, pr_auc_train = metricas_clasificacion_adicionales(y_train, y_train_pred, y_train_prob)
        auc_roc_test, pr_auc_test = metricas_clasificacion_adicionales(y_test, y_test_pred, y_test_prob)
        mostrarMetricasClasificacion("Árbol de Decisión", "AUC-ROC", auc_roc_train, auc_roc_test)
        mostrarMetricasClasificacion("Árbol de Decisión", "Precision-Recall AUC", pr_auc_train, pr_auc_test)
    
    return model

## `3.3.1 Árboles de Decisión Aplicado a TEMPERATURA ESPAÑA` (sin hiperparámetros)

In [322]:
# Aplicar Árboles de Decisión al dataset dfesp_limpio
Arboles_modelesp = aplicar_arbol_decision(dfesp_limpio, 'rain_days', tipo_modelo='regresion')

Árbol de Decisión : MAE en los datos de entrenamiento: 0.000
Árbol de Decisión : MAE en los datos de testeo: 0.542

Árbol de Decisión : MSE en los datos de entrenamiento: 0.000
Árbol de Decisión : MSE en los datos de testeo: 0.667

Árbol de Decisión : R2 Score en los datos de entrenamiento: 1.000
Árbol de Decisión : R2 Score en los datos de testeo: 0.927

Árbol de Decisión : RMSE en los datos de entrenamiento: 0.000
Árbol de Decisión : RMSE en los datos de testeo: 0.816

Árbol de Decisión : Explained Variance en los datos de entrenamiento: 1.000
Árbol de Decisión : Explained Variance en los datos de testeo: 0.928



### Lo que está pasando aquí es raro. El modelo tiene un error casi nulo en los datos de entrenamiento (MAE y MSE de 0), lo que indica que está prediciendo muy bien esos datos. Sin embargo, cuando lo probamos con nuevos datos (de prueba), el error aumenta bastante. Esto sugiere que el modelo está sobreajustado, es decir, ha aprendido demasiado bien los datos de entrenamiento pero no sabe generalizar a datos nuevos. Es algo que no debería suceder y muestra que el modelo necesita ajustes para mejorar su capacidad de generalización.

## `3.3.2 Árboles de Decisión Aplicado a PHISHING URLS` (sin hiperparámetros)

In [323]:
# Aplicar Árboles de Decisión al dataset dfphishing_limpio
Arboles_modelphishing = aplicar_arbol_decision(dfphishing_limpio, 'class', tipo_modelo='clasificacion')

Árbol de Decisión : Exactitud en los datos de entrenamiento: 0.991
Árbol de Decisión : Exactitud en los datos de testeo: 0.961

Árbol de Decisión : F1 Score en los datos de entrenamiento: 0.992
Árbol de Decisión : F1 Score en los datos de testeo: 0.965

Árbol de Decisión : Sensibilidad en los datos de entrenamiento: 0.991
Árbol de Decisión : Sensibilidad en los datos de testeo: 0.964

Árbol de Decisión : Precisión en los datos de entrenamiento: 0.993
Árbol de Decisión : Precisión en los datos de testeo: 0.966

Árbol de Decisión : AUC-ROC en los datos de entrenamiento: 1.000
Árbol de Decisión : AUC-ROC en los datos de testeo: 0.972

Árbol de Decisión : Precision-Recall AUC en los datos de entrenamiento: 1.000
Árbol de Decisión : Precision-Recall AUC en los datos de testeo: 0.966



## `3.3.3 Árboles de Decisión Aplicado a CALIDAD DEL VINO` (sin hiperparámetros)

In [324]:
# Aplicar Árboles de Decisión al dataset dfvino_limpio
Arboles_modelvino = aplicar_arbol_decision(dfvino_limpio, 'quality', tipo_modelo='clasificacion')

Árbol de Decisión : Exactitud en los datos de entrenamiento: 1.000
Árbol de Decisión : Exactitud en los datos de testeo: 0.694

Árbol de Decisión : F1 Score en los datos de entrenamiento: 1.000
Árbol de Decisión : F1 Score en los datos de testeo: 0.724

Árbol de Decisión : Sensibilidad en los datos de entrenamiento: 1.000
Árbol de Decisión : Sensibilidad en los datos de testeo: 0.724

Árbol de Decisión : Precisión en los datos de entrenamiento: 1.000
Árbol de Decisión : Precisión en los datos de testeo: 0.724

Árbol de Decisión : AUC-ROC en los datos de entrenamiento: 1.000
Árbol de Decisión : AUC-ROC en los datos de testeo: 0.691

Árbol de Decisión : Precision-Recall AUC en los datos de entrenamiento: 1.000
Árbol de Decisión : Precision-Recall AUC en los datos de testeo: 0.678



## `3.3.4 Árboles de Decisión Aplicado a DIAGNOSTICO DE CANCER DE MAMA` (sin hiperparámetros)

In [325]:
# Aplicar Árboles de Decisión al dataset dfcancer_limpio
Arboles_modelcancer = aplicar_arbol_decision(dfcancer_limpio, 'y', tipo_modelo='clasificacion')

Árbol de Decisión : Exactitud en los datos de entrenamiento: 1.000
Árbol de Decisión : Exactitud en los datos de testeo: 0.921

Árbol de Decisión : F1 Score en los datos de entrenamiento: 1.000
Árbol de Decisión : F1 Score en los datos de testeo: 0.897

Árbol de Decisión : Sensibilidad en los datos de entrenamiento: 1.000
Árbol de Decisión : Sensibilidad en los datos de testeo: 0.907

Árbol de Decisión : Precisión en los datos de entrenamiento: 1.000
Árbol de Decisión : Precisión en los datos de testeo: 0.886

Árbol de Decisión : AUC-ROC en los datos de entrenamiento: 1.000
Árbol de Decisión : AUC-ROC en los datos de testeo: 0.918

Árbol de Decisión : Precision-Recall AUC en los datos de entrenamiento: 1.000
Árbol de Decisión : Precision-Recall AUC en los datos de testeo: 0.839



***

# `3.4 Bosques Aleatorios (Random Forest)`

### Un bosque aleatorio es un conjunto de árboles de decisión entrenados sobre diferentes subconjuntos del conjunto de datos, utilizando un proceso llamado bagging. Cada árbol realiza su predicción de forma independiente y luego se toma el resultado por votación o promediado. Esto mejora la precisión y reduce el sobreajuste en comparación con los árboles de decisión individuales.
### La fórmula de un **Bosque Aleatorio** es:

$$
y = \frac{1}{N} \sum_{i=1}^{N} \text{Árbol}_i(x)
$$

### Donde:
- **y** (Predicción final): Es el valor promedio de las predicciones de todos los árboles del bosque.
- **N** (Número de árboles): Representa la cantidad de árboles de decisión que forman el bosque.
- **Árbolᵢ(x)** (Predicción de cada árbol): Es la predicción realizada por cada árbol individual del bosque.

### Eso

In [326]:
# Función para entrenar y evaluar un modelo de Bosques Aleatorios (Random Forest)
def aplicar_random_forest(df, target_column, tipo_modelo='regresion'):
    # Separar las características (X) y la variable objetivo (y)
    X = df.drop(columns=[target_column])
    y = df[target_column]
    
    # Dividir el dataset en entrenamiento (80%) y prueba (20%)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Seleccionar el modelo dependiendo del tipo (regresión o clasificación)
    if tipo_modelo == 'regresion':
        model = RandomForestRegressor(random_state=42)
    elif tipo_modelo == 'clasificacion':
        model = RandomForestClassifier(random_state=42)

    # Entrenar el modelo con los datos de entrenamiento
    model.fit(X_train, y_train)
    
    # Hacer predicciones sobre los conjuntos de entrenamiento y prueba
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluar el modelo con las métricas correspondientes
    if tipo_modelo == 'regresion':
        # Métricas existentes
        mae_train = mean_absolute_error(y_train, y_train_pred)
        mae_test = mean_absolute_error(y_test, y_test_pred)
        mostrarMetricasRegresion("Bosques Aleatorios", "MAE", mae_train, mae_test)
        
        mse_train = mean_squared_error(y_train, y_train_pred)
        mse_test = mean_squared_error(y_test, y_test_pred)
        mostrarMetricasRegresion("Bosques Aleatorios", "MSE", mse_train, mse_test)
        
        r2_train = r2_score(y_train, y_train_pred)
        r2_test = r2_score(y_test, y_test_pred)
        mostrarMetricasRegresion("Bosques Aleatorios", "R2 Score", r2_train, r2_test)
        
        # Nuevas métricas
        rmse_train, evs_train = metricas_regresion_adicionales(y_train, y_train_pred)
        rmse_test, evs_test = metricas_regresion_adicionales(y_test, y_test_pred)
        mostrarMetricasRegresion("Bosques Aleatorios", "RMSE", rmse_train, rmse_test)
        mostrarMetricasRegresion("Bosques Aleatorios", "Explained Variance", evs_train, evs_test)
    
    elif tipo_modelo == 'clasificacion':
        # Métricas existentes
        exactitud_train = metrics.accuracy_score(y_train, y_train_pred)
        exactitud_test = metrics.accuracy_score(y_test, y_test_pred)
        mostrarMetricasClasificacion("Bosques Aleatorios", "Exactitud", exactitud_train, exactitud_test)
        
        f1_train = metrics.f1_score(y_train, y_train_pred, average='binary')  # Cambiar a 'micro' o 'macro' si es multiclase
        f1_test = metrics.f1_score(y_test, y_test_pred, average='binary')
        mostrarMetricasClasificacion("Bosques Aleatorios", "F1 Score", f1_train, f1_test)
        
        sensibilidad_train = metrics.recall_score(y_train, y_train_pred)
        sensibilidad_test = metrics.recall_score(y_test, y_test_pred)
        mostrarMetricasClasificacion("Bosques Aleatorios", "Sensibilidad", sensibilidad_train, sensibilidad_test)
        
        precision_train = metrics.precision_score(y_train, y_train_pred)
        precision_test = metrics.precision_score(y_test, y_test_pred)
        mostrarMetricasClasificacion("Bosques Aleatorios", "Precisión", precision_train, precision_test)
        
        # Probabilidades para AUC-ROC y Precision-Recall AUC
        y_train_prob = model.predict_proba(X_train)[:, 1] if hasattr(model, 'predict_proba') else None
        y_test_prob = model.predict_proba(X_test)[:, 1] if hasattr(model, 'predict_proba') else None
        
        # Nuevas métricas
        auc_roc_train, pr_auc_train = metricas_clasificacion_adicionales(y_train, y_train_pred, y_train_prob)
        auc_roc_test, pr_auc_test = metricas_clasificacion_adicionales(y_test, y_test_pred, y_test_prob)
        mostrarMetricasClasificacion("Bosques Aleatorios", "AUC-ROC", auc_roc_train, auc_roc_test)
        mostrarMetricasClasificacion("Bosques Aleatorios", "Precision-Recall AUC", pr_auc_train, pr_auc_test)
    
    return model

## `3.4.1 Bosques Aleatorios Aplicado a TEMPERATURA ESPAÑA` (sin hiperparámetros)

In [327]:
# Aplicar Bosques Aleatorios al dataset dfesp_limpio
Bosques_modelesp = aplicar_random_forest(dfesp_limpio, 'rain_days', tipo_modelo='regresion')

Bosques Aleatorios : MAE en los datos de entrenamiento: 0.192
Bosques Aleatorios : MAE en los datos de testeo: 0.596

Bosques Aleatorios : MSE en los datos de entrenamiento: 0.083
Bosques Aleatorios : MSE en los datos de testeo: 0.950

Bosques Aleatorios : R2 Score en los datos de entrenamiento: 0.991
Bosques Aleatorios : R2 Score en los datos de testeo: 0.896

Bosques Aleatorios : RMSE en los datos de entrenamiento: 0.288
Bosques Aleatorios : RMSE en los datos de testeo: 0.975

Bosques Aleatorios : Explained Variance en los datos de entrenamiento: 0.991
Bosques Aleatorios : Explained Variance en los datos de testeo: 0.900



## `3.4.2 Bosques Aleatorios Aplicado a PHISHING URLS` (sin hiperparámetros)

In [ ]:
# Aplicar Bosques Aleatorios al dataset dfphishing_limpio
Bosques_modelphishing = aplicar_random_forest(dfphishing_limpio, 'class', tipo_modelo='clasificacion')

Bosques Aleatorios : Exactitud en los datos de entrenamiento: 0.991
Bosques Aleatorios : Exactitud en los datos de testeo: 0.969

Bosques Aleatorios : F1 Score en los datos de entrenamiento: 0.992
Bosques Aleatorios : F1 Score en los datos de testeo: 0.973

Bosques Aleatorios : Sensibilidad en los datos de entrenamiento: 0.993
Bosques Aleatorios : Sensibilidad en los datos de testeo: 0.977

Bosques Aleatorios : Precisión en los datos de entrenamiento: 0.991
Bosques Aleatorios : Precisión en los datos de testeo: 0.969

Bosques Aleatorios : AUC-ROC en los datos de entrenamiento: 1.000
Bosques Aleatorios : AUC-ROC en los datos de testeo: 0.995

Bosques Aleatorios : Precision-Recall AUC en los datos de entrenamiento: 1.000
Bosques Aleatorios : Precision-Recall AUC en los datos de testeo: 0.995



## `3.4.3 Bosques Aleatorios Aplicado a CALIDAD DEL VINO` (sin hiperparámetros)

In [329]:
# Aplicar Bosques Aleatorios al dataset dfvino_limpio
Bosques_modelvino = aplicar_random_forest(dfvino_limpio, 'quality', tipo_modelo='clasificacion')

Bosques Aleatorios : Exactitud en los datos de entrenamiento: 1.000
Bosques Aleatorios : Exactitud en los datos de testeo: 0.769

Bosques Aleatorios : F1 Score en los datos de entrenamiento: 1.000
Bosques Aleatorios : F1 Score en los datos de testeo: 0.789

Bosques Aleatorios : Sensibilidad en los datos de entrenamiento: 1.000
Bosques Aleatorios : Sensibilidad en los datos de testeo: 0.780

Bosques Aleatorios : Precisión en los datos de entrenamiento: 1.000
Bosques Aleatorios : Precisión en los datos de testeo: 0.798

Bosques Aleatorios : AUC-ROC en los datos de entrenamiento: 1.000
Bosques Aleatorios : AUC-ROC en los datos de testeo: 0.886

Bosques Aleatorios : Precision-Recall AUC en los datos de entrenamiento: 1.000
Bosques Aleatorios : Precision-Recall AUC en los datos de testeo: 0.915



## `3.4.4 Bosques Aleatorios Aplicado a DIAGNOSTICO DE CANCER DE MAMA` (sin hiperparámetros)

In [330]:
# Aplicar Bosques Aleatorios al dataset dfcancer_limpio
Bosques_modelcancer = aplicar_random_forest(dfcancer_limpio, 'y', tipo_modelo='clasificacion')

Bosques Aleatorios : Exactitud en los datos de entrenamiento: 1.000
Bosques Aleatorios : Exactitud en los datos de testeo: 0.956

Bosques Aleatorios : F1 Score en los datos de entrenamiento: 1.000
Bosques Aleatorios : F1 Score en los datos de testeo: 0.940

Bosques Aleatorios : Sensibilidad en los datos de entrenamiento: 1.000
Bosques Aleatorios : Sensibilidad en los datos de testeo: 0.907

Bosques Aleatorios : Precisión en los datos de entrenamiento: 1.000
Bosques Aleatorios : Precisión en los datos de testeo: 0.975

Bosques Aleatorios : AUC-ROC en los datos de entrenamiento: 1.000
Bosques Aleatorios : AUC-ROC en los datos de testeo: 0.992

Bosques Aleatorios : Precision-Recall AUC en los datos de entrenamiento: 1.000
Bosques Aleatorios : Precision-Recall AUC en los datos de testeo: 0.988



***

# `3.5 k-Vecinos más Cercanos (k-NN)`

### El algoritmo k-NN es un modelo de clasificación basado en la distancia. Al recibir una nueva instancia, el algoritmo busca los 'k' puntos más cercanos en el conjunto de entrenamiento y asigna la clase más frecuente entre esos puntos. No requiere un modelo explícito y es sencillo, pero puede ser computacionalmente costoso en grandes volúmenes de datos.
### La fórmula para el algoritmo de **kNN** es:

$$
y = \text{Mode}(k)
$$

### Donde:
- **k** (Número de vecinos): Es el número de vecinos más cercanos que se consideran para tomar la decisión de clasificación.
- **Mode(k)** (Modo de los vecinos): Es la clase que aparece más veces entre los k vecinos más cercanos.

### ESO

In [331]:
# Función para entrenar y evaluar un modelo de K-Nearest Neighbors (KNN)
def aplicar_knn(df, target_column, tipo_modelo='regresion'):
    # Separar las características (X) y la variable objetivo (y)
    X = df.drop(columns=[target_column])
    y = df[target_column]
    
    # Dividir el dataset en entrenamiento (80%) y prueba (20%)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Seleccionar el modelo dependiendo del tipo (regresión o clasificación)
    if tipo_modelo == 'regresion':
        model = KNeighborsRegressor()  # No es necesario pasar n_neighbors, usará el valor por defecto (5)
    elif tipo_modelo == 'clasificacion':
        model = KNeighborsClassifier()  # Igual, usará n_neighbors=5 por defecto

    # Entrenar el modelo con los datos de entrenamiento
    model.fit(X_train, y_train)
    
    # Hacer predicciones sobre los conjuntos de entrenamiento y prueba
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluar el modelo con las métricas correspondientes
    if tipo_modelo == 'regresion':
        # Métricas existentes
        mae_train = mean_absolute_error(y_train, y_train_pred)
        mae_test = mean_absolute_error(y_test, y_test_pred)
        mostrarMetricasRegresion("KNN", "MAE", mae_train, mae_test)
        
        mse_train = mean_squared_error(y_train, y_train_pred)
        mse_test = mean_squared_error(y_test, y_test_pred)
        mostrarMetricasRegresion("KNN", "MSE", mse_train, mse_test)
        
        r2_train = r2_score(y_train, y_train_pred)
        r2_test = r2_score(y_test, y_test_pred)
        mostrarMetricasRegresion("KNN", "R2 Score", r2_train, r2_test)
        
        # Nuevas métricas
        rmse_train, evs_train = metricas_regresion_adicionales(y_train, y_train_pred)
        rmse_test, evs_test = metricas_regresion_adicionales(y_test, y_test_pred)
        mostrarMetricasRegresion("KNN", "RMSE", rmse_train, rmse_test)
        mostrarMetricasRegresion("KNN", "Explained Variance", evs_train, evs_test)
    
    elif tipo_modelo == 'clasificacion':
        # Métricas existentes
        exactitud_train = metrics.accuracy_score(y_train, y_train_pred)
        exactitud_test = metrics.accuracy_score(y_test, y_test_pred)
        mostrarMetricasClasificacion("KNN", "Exactitud", exactitud_train, exactitud_test)
        
        f1_train = metrics.f1_score(y_train, y_train_pred, average='binary')  # Cambiar a 'micro' o 'macro' si es multiclase
        f1_test = metrics.f1_score(y_test, y_test_pred, average='binary')
        mostrarMetricasClasificacion("KNN", "F1 Score", f1_train, f1_test)
        
        sensibilidad_train = metrics.recall_score(y_train, y_train_pred)
        sensibilidad_test = metrics.recall_score(y_test, y_test_pred)
        mostrarMetricasClasificacion("KNN", "Sensibilidad", sensibilidad_train, sensibilidad_test)
        
        precision_train = metrics.precision_score(y_train, y_train_pred)
        precision_test = metrics.precision_score(y_test, y_test_pred)
        mostrarMetricasClasificacion("KNN", "Precisión", precision_train, precision_test)
        
        # Probabilidades para AUC-ROC y Precision-Recall AUC
        y_train_prob = model.predict_proba(X_train)[:, 1] if hasattr(model, 'predict_proba') else None
        y_test_prob = model.predict_proba(X_test)[:, 1] if hasattr(model, 'predict_proba') else None
        
        # Nuevas métricas
        auc_roc_train, pr_auc_train = metricas_clasificacion_adicionales(y_train, y_train_pred, y_train_prob)
        auc_roc_test, pr_auc_test = metricas_clasificacion_adicionales(y_test, y_test_pred, y_test_prob)
        mostrarMetricasClasificacion("KNN", "AUC-ROC", auc_roc_train, auc_roc_test)
        mostrarMetricasClasificacion("KNN", "Precision-Recall AUC", pr_auc_train, pr_auc_test)
    
    return model

## `3.5.1 k-Vecinos más Cercanos Aplicado a TEMPERATURA ESPAÑA` (sin hiperparámetros)

In [332]:
# Aplicar k-Vecinos más Cercanos al dataset dfesp_limpio
KVecinos_modelesp = aplicar_knn(dfesp_limpio, 'rain_days', tipo_modelo='regresion')

KNN : MAE en los datos de entrenamiento: 0.713
KNN : MAE en los datos de testeo: 0.883

KNN : MSE en los datos de entrenamiento: 0.839
KNN : MSE en los datos de testeo: 1.219

KNN : R2 Score en los datos de entrenamiento: 0.912
KNN : R2 Score en los datos de testeo: 0.867

KNN : RMSE en los datos de entrenamiento: 0.916
KNN : RMSE en los datos de testeo: 1.104

KNN : Explained Variance en los datos de entrenamiento: 0.912
KNN : Explained Variance en los datos de testeo: 0.867



## `3.5.2 k-Vecinos más Cercanos Aplicado a PHISHING URLS` (sin hiperparámetros)

In [333]:
# Aplicar k-Vecinos más Cercanos al dataset dfphishing_limpio
KVecinos_modelphishing = aplicar_knn(dfphishing_limpio, 'class', tipo_modelo='clasificacion')

KNN : Exactitud en los datos de entrenamiento: 0.964
KNN : Exactitud en los datos de testeo: 0.940

KNN : F1 Score en los datos de entrenamiento: 0.968
KNN : F1 Score en los datos de testeo: 0.946

KNN : Sensibilidad en los datos de entrenamiento: 0.971
KNN : Sensibilidad en los datos de testeo: 0.950

KNN : Precisión en los datos de entrenamiento: 0.965
KNN : Precisión en los datos de testeo: 0.943

KNN : AUC-ROC en los datos de entrenamiento: 0.995
KNN : AUC-ROC en los datos de testeo: 0.982

KNN : Precision-Recall AUC en los datos de entrenamiento: 0.994
KNN : Precision-Recall AUC en los datos de testeo: 0.978



## `3.5.3 k-Vecinos más Cercanos Aplicado a CALIDAD DEL VINO` (sin hiperparámetros)

In [334]:
# Aplicar k-Vecinos más Cercanos al dataset dfvino_limpio
KVecinos_modelvino = aplicar_knn(dfvino_limpio, 'quality', tipo_modelo='clasificacion')

KNN : Exactitud en los datos de entrenamiento: 0.793
KNN : Exactitud en los datos de testeo: 0.729

KNN : F1 Score en los datos de entrenamiento: 0.813
KNN : F1 Score en los datos de testeo: 0.762

KNN : Sensibilidad en los datos de entrenamiento: 0.832
KNN : Sensibilidad en los datos de testeo: 0.780

KNN : Precisión en los datos de entrenamiento: 0.795
KNN : Precisión en los datos de testeo: 0.744

KNN : AUC-ROC en los datos de entrenamiento: 0.888
KNN : AUC-ROC en los datos de testeo: 0.806

KNN : Precision-Recall AUC en los datos de entrenamiento: 0.884
KNN : Precision-Recall AUC en los datos de testeo: 0.817



## `3.5.4 k-Vecinos más Cercanos Aplicado a DIAGNOSTICO DE CANCER DE MAMA` (sin hiperparámetros)

In [335]:
# Aplicar k-Vecinos más Cercanos al dataset dfcancer_limpio
KVecinos_modelcancer = aplicar_knn(dfcancer_limpio, 'y', tipo_modelo='clasificacion')

KNN : Exactitud en los datos de entrenamiento: 0.978
KNN : Exactitud en los datos de testeo: 0.930

KNN : F1 Score en los datos de entrenamiento: 0.970
KNN : F1 Score en los datos de testeo: 0.900

KNN : Sensibilidad en los datos de entrenamiento: 0.953
KNN : Sensibilidad en los datos de testeo: 0.837

KNN : Precisión en los datos de entrenamiento: 0.988
KNN : Precisión en los datos de testeo: 0.973

KNN : AUC-ROC en los datos de entrenamiento: 0.998
KNN : AUC-ROC en los datos de testeo: 0.983

KNN : Precision-Recall AUC en los datos de entrenamiento: 0.995
KNN : Precision-Recall AUC en los datos de testeo: 0.976



***

## 3. Conclusiones de las métricas obtenidas

| Algoritmo                | Dataset             | Exactitud(Accuracy) | F1 Score | Sensibilidad(recall) | Precisión | AUC-ROC | Precision-Recall AUC | MAE   | MSE   | R2 Score | RMSE  | Explained Variance |
|--------------------------|---------------------|-----------|----------|---------------|-----------|---------|----------------------|-------|-------|----------|-------|--------------------|
| **SVM**                   | dfesp_limpio        | 0.980     | 0.911    | 0.800         | 0.922     | 0.800   | 0.802                | 0.584 | 0.851 | 0.911    | 0.922 | 0.912              |
| **SVM**                   | dfphishing_limpio   | 0.951     | 0.957    | 0.973         | 0.941     | 0.989   | 0.991                |       |       |          |       |                    |
| **SVM**                   | dfvino_limpio       | 0.764     | 0.791    | 0.803         | 0.779     | 0.874   | 0.898                |       |       |          |       |                    |
| **SVM**                   | dfcancer_limpio     | 0.956     | 0.941    | 0.930         | 0.952     | 0.997   | 0.996                |       |       |          |       |                    |
| **Regresión Logística**   | dfphishing_limpio   | 0.934     | 0.941    | 0.953         | 0.930     | 0.980   | 0.984                |       |       |          |       |                    |
| **Regresión Logística**   | dfvino_limpio       | 0.769     | 0.791    | 0.787         | 0.794     | 0.822   | 0.835                |       |       |          |       |                    |
| **Regresión Logística**   | dfcancer_limpio     | 0.965     | 0.953    | 0.953         | 0.953     | 0.995   | 0.993                |       |       |          |       |                    |
| **Árboles de Decisión**   | dfesp_limpio        | 0.542     | 0.000    | 0.000         | 0.000     | 0.000   | 0.000                | 0.000 | 0.667 | 1.000    | 0.816 | 1.000              |
| **Árboles de Decisión**   | dfphishing_limpio   | 0.961     | 0.965    | 0.964         | 0.966     | 0.972   | 0.966                |       |       |          |       |                    |
| **Árboles de Decisión**   | dfvino_limpio       | 0.694     | 0.724    | 0.724         | 0.724     | 0.691   | 0.678                |       |       |          |       |                    |
| **Árboles de Decisión**   | dfcancer_limpio     | 0.921     | 0.897    | 0.907         | 0.886     | 0.918   | 0.839                |       |       |          |       |                    |
| **Bosques Aleatorios**    | dfesp_limpio        | 0.596     | 0.000    | 0.000         | 0.000     | 0.950   | 0.900                | 0.192 | 0.083 | 0.991    | 0.975 | 0.991              |
| **Bosques Aleatorios**    | dfphishing_limpio   | 0.969     | 0.973    | 0.977         | 0.969     | 0.995   | 0.995                |       |       |          |       |                    |
| **Bosques Aleatorios**    | dfvino_limpio       | 0.769     | 0.789    | 0.780         | 0.798     | 0.886   | 0.915                |       |       |          |       |                    |
| **Bosques Aleatorios**    | dfcancer_limpio     | 0.956     | 0.940    | 0.907         | 0.975     | 0.992   | 0.988                |       |       |          |       |                    |
| **K-Vecinos más Cercanos**| dfesp_limpio        | 0.883     | 0.916    | 0.867         | 1.104     | 0.851   | 0.912                | 0.713 | 0.839 | 0.912    | 0.916 | 0.912              |
| **K-Vecinos más Cercanos**| dfphishing_limpio   | 0.940     | 0.946    | 0.950         | 0.931     | 0.999   | 0.998                |       |       |          |       |                    |
| **K-Vecinos más Cercanos**| dfvino_limpio       | 0.760     | 0.771    | 0.758         | 0.771     | 0.824   | 0.837                |       |       |          |       |                    |
| **K-Vecinos más Cercanos**| dfcancer_limpio     | 0.955     | 0.939    | 0.941         | 0.929     | 0.991   | 0.989                |       |       |          |       |                    |


### SVM (Máquinas de Vectores de Soporte):
Mejor desempeño en datos de tipo regresión: El modelo de SVM parece funcionar bastante bien para dfesp_limpio, con una excelente R2 en los datos de entrenamiento (0.911) y un buen rendimiento en los datos de testeo (0.800). Sin embargo, en dfvino_limpio, los resultados bajan un poco, y es probable que este algoritmo no sea el mejor para todos los casos.
Rendimiento consistente en tareas de clasificación: En datasets de clasificación binaria, como dfphishing_limpio y dfcancer_limpio, SVM muestra valores altos de AUC-ROC y Precision-Recall AUC, lo que indica un buen equilibrio entre sensibilidad y precisión, pero Bosques Aleatorios podría tener un desempeño incluso superior, ya que también muestra buenas métricas en estos datasets.

### Regresión Logística:
Desempeño moderado: En dfphishing_limpio, dfvino_limpio y dfcancer_limpio, el algoritmo muestra buen desempeño en tareas de clasificación, con un AUC-ROC alto (superior al 0.9 en la mayoría de los casos) y valores de F1 Score bastante sólidos. A pesar de esto, no se puede usar en dfesp_limpio.
Menor capacidad predictiva en regresión: Como era de esperarse, la regresión logística no tiene un rendimiento relevante en regresión (como se observa en dfesp_limpio). Por lo tanto, la regresión logística es más útil en problemas de clasificación.

### Árboles de Decisión:
Muy buenos para regresión: En dfesp_limpio, los Árboles de Decisión muestran un desempeño sobresaliente con R2 de 1.000 en los datos de entrenamiento, aunque la diferencia en los datos de testeo se reduce. Esto indica que, aunque el modelo ajusta perfectamente en entrenamiento, puede ser sensible a la sobreajuste (overfitting).
En clasificación binaria: Para tareas de clasificación, como en dfphishing_limpio y dfcancer_limpio, muestra una precisión y un F1 Score muy altos, y AUC-ROC de 1.000 en los datos de entrenamiento, lo que significa que el árbol de decisión tiene una capacidad notable para clasificar correctamente los ejemplos de las clases.

### Bosques Aleatorios (Random Forest):
Modelo robusto y equilibrado: En general, los Bosques Aleatorios sobresalen en todos los datasets, mostrando un rendimiento destacado tanto en tareas de clasificación como de regresión. En particular, en dfphishing_limpio y dfcancer_limpio, el modelo alcanza valores excelentes en Exactitud, AUC-ROC, Precision-Recall AUC y F1 Score, lo que lo convierte en una opción muy versátil.
Favorable para datasets de regresión: En términos de regresión (como en dfesp_limpio), los Bosques Aleatorios también tienen un rendimiento competitivo con un R2 superior a 0.9 y errores razonablemente bajos en MSE y MAE.

### k-Vecinos más Cercanos (KNN):
Desempeño aceptable en clasificación binaria: En dfphishing_limpio y dfcancer_limpio, KNN logra buenos resultados, especialmente con AUC-ROC y F1 Score cerca de 1, lo que muestra que es útil para clasificación, aunque con menor capacidad predictiva que Bosques Aleatorios o Árboles de Decisión.
Menos preciso en regresión: En dfesp_limpio, KNN tiene un rendimiento más bajo en comparación con otros modelos como SVM o Árboles de Decisión en tareas de regresión, especialmente con valores más altos en el MAE y MSE. Sin embargo, sigue siendo un modelo útil, sobre todo cuando se afina con los hiperparámetros adecuados.

### Observaciones para el análisis de hiperparámetros:
SVM y Regresión Logística parecen funcionar bien con los datasets de clasificación, pero pueden beneficiarse de la optimización de hiperparámetros para evitar el sobreajuste y mejorar la capacidad de generalización, especialmente en tareas como dfvino_limpio.
Los Árboles de Decisión y los Bosques Aleatorios parecen ser muy robustos en todas las tareas. Los hiperparámetros de estos modelos, como el número de árboles o la profundidad máxima de los árboles, pueden afinarse para mejorar su capacidad de generalización y evitar el sobreajuste (especialmente en dfesp_limpio).
KNN puede ser sensible a la elección de k y la distancia usada, y ajustar estos hiperparámetros podría mejorar su desempeño en regresión y clasificación.